# STGCN

This folder follows the same structure of the files that lie in the [main](https://github.com/VeritasYin/STGCN_IJCAI-18) Github repository. Many directories not directly relevant to STGCN training have been omitted. The STGCN is loaded on Google Drive and can be trained [here](https://drive.google.com/file/d/17spvTDAU4Le-15dqlv6hhmARRofuVJO1/view?usp=sharing) to make use of Colab’s GPUs.

# Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In order to run this notebook, you will have to request access to [this](https://drive.google.com/drive/folders/1OrNCjHFZ71M5OiIQtu9G7C61kFl8aVGT?usp=sharing) Google Drive folder, copy into your Drive, and change the path below.

In [ ]:
import os
aman_path_to_capstone_dir = 'drive/MyDrive/School/Undergrad/Spring 2022/Capstone/Models/' 
os.chdir(f'{aman_path_to_capstone_dir}STGCN Training/models')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
!pip3 install prophet
import sys
sys.path.append('/content/drive/MyDrive/School/Undergrad/Spring 2022/Capstone/Models/STGCN Training/utils/')
from utils import load_processed_data

### Format

In [ ]:
adj_mat, ind_station_mapper, speeds = load_processed_data('../data/processed/fwy_405_n_ds')

In [ ]:
import yaml

with open('../models/env.yaml') as f:
    ENV = yaml.load(f, Loader=yaml.Loader)

In [ ]:
# subset and choose data in may-july
speeds = speeds[speeds.index.month.isin([5, 6, 7])] 

# write to file
outfile = '../data/processed/fwy_405_n_ds/speeds_form.csv'
speeds.to_csv(outfile, index=False, header=False)

# write to file
outfile = '../data/processed/fwy_405_n_ds/adj_mat_form.csv'
pd.DataFrame(adj_mat).to_csv(outfile, index=False, header=False)

# Build Model

In [ ]:
%cd ../utils/STGCN

In [ ]:
!python main.py --epoch=25 --save=1

# Evaluate

In [ ]:
actual = np.load('../../models/trained/STGCN/preds/actual.npy')
predicted = np.load('../../models/trained/STGCN/preds/predicted.npy')

In [ ]:
# helper code to figure out what timestamps were used in the test set
new = [[x[0] for x in y] for y in actual]
new = np.array(new)

seq = new[0:100,5]

full = speeds.iloc[:,5].values

for i in range(len(full) - len(seq)):
    if (full[i:i+len(seq)] == seq).all():
        print(i)


In [ ]:
# format into dataframes
ts_start = 11246

actual_df = pd.DataFrame([[x[0] for x in y] for y in actual])
actual_df.columns = speeds.columns
actual_df.index = speeds.index[ts_start:ts_start+new.shape[0]]

predicted_df = pd.DataFrame([[x[0] for x in y] for y in predicted])
predicted_df.columns = speeds.columns
predicted_df.index = speeds.index[ts_start:ts_start+new.shape[0]]

print(predicted_df.shape)
predicted_df.head()

In [ ]:
import plotly.graph_objects as go

In [ ]:
station = 771826
cutoff = int(0.75 * actual_df.index.shape[0])

fig = go.Figure()
fig.add_trace(go.Line(x=actual_df.index, y=actual_df[station], name='True Values'))
fig.add_trace(go.Line(x=predicted_df.index[:cutoff], y=predicted_df.loc[:(predicted_df.index[cutoff]), station], name='Predicted Values (Train)'))
fig.add_trace(go.Line(x=predicted_df.index[cutoff:], y=predicted_df.loc[(predicted_df.index[cutoff]):, station], name='Predicted Values (Test)'))
fig.update_layout(
    title="STGCN Forecast Results",
    xaxis_title="Time",
    yaxis_title="Forecast")

In [ ]:
# fig.write_html('../plots/STGCN.html')